In [22]:
import numpy as np
from dowhy import CausalModel
import dowhy.datasets
import dowhy.api

In [4]:
# Load some sample data
data = dowhy.datasets.linear_dataset(
    beta=10,
    num_common_causes=5,
    num_instruments=2,
    num_samples=10000,
    treatment_is_binary=True)

In [7]:
data['df']

,Z0,Z1,W0,W1,W2,W3,W4,v0,y
0,0.0,0.479982,-1.055416,0.962645,0.246433,1.182173,0.301819,True,12.193166
1,1.0,0.717892,-0.798479,0.554180,0.414186,0.819138,-0.147000,True,10.343629
2,1.0,0.226585,-0.152804,-0.042631,0.590433,-0.472660,0.808104,True,14.280896
3,1.0,0.610615,-0.100076,-1.647035,1.923981,0.681609,1.065725,True,21.873577
4,1.0,0.701913,0.100276,2.225072,4.509363,0.810958,1.473980,True,32.825955
...,...,...,...,...,...,...,...,...,...
9995,1.0,0.592356,-0.815404,-0.900140,0.424413,0.025788,1.809355,True,17.921251
9996,1.0,0.042535,-0.687013,-1.448990,0.058578,0.120764,-1.588664,False,-9.139777
9997,1.0,0.183571,0.929452,2.335344,-0.067710,2.107024,1.010953,True,21.751011
9998,1.0,0.761843,0.016831,-0.002846,3.012534,0.602294,1.406407,True,27.100252


In [23]:
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

FileNotFoundError: [Errno 2] No such file or directory: 'causal_model.png'

In [9]:
model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['v0'] on outcome ['y']


In [11]:
model.view_model()

In [12]:
identified_estimand = model.identify_effect()

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['W4', 'Unobserved Confounders', 'W1', 'W2', 'W3', 'W0']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:['Z1', 'Z0']


In [13]:
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W1+W2+W3+W0
/Users/cyanny/Codecookies/infra-wp/python-wp/py3env/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y~v0+W4+W1+W2+W3+W0+w_random
/Users/cyanny/Codecookies/infra-wp/python-wp/py3env/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
